# Nearest Neighbor Decoder Demo

This notebook demonstrates how to use the `NearestNeighborDecoder` to decode received vectors using the nearest neighbor strategy.  
We use a binary linear $[7,4]$ code over $\mathbb{F}_2$ with the following generator matrix:

$$
G = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1
\end{bmatrix}
$$

We will test the decoder on both valid and noisy inputs, comparing received vectors to all codewords and returning the closest match.

In [1]:
# Imports
import numpy as np
import sys
import os
sys.path.append(os.path.abspath("../src"))
from decoders import NearestNeighborDecoder

## Define the Generator Matrix

We use the following generator matrix $G$ for our $[7,4]$ binary code. Each row corresponds to a basis vector for the code.

In [2]:
# Define the generator matrix for a [3,2] binary code
G = [
    [1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1]
]

## Instantiate the Nearest Neighbor Decoder

We create an instance of the `NearestNeighborDecoder` using the generator matrix $G$ and field size $p=2$.

In [3]:
# Instantiate the decoder
decoder = NearestNeighborDecoder(G, p=2)

## Code Parameters

Let's display the parameters of our $[n, k]$ code:
- $n$: codeword length
- $k$: message length

In [4]:
decoder.display_n()
decoder.display_k()

n (codeword length): 7
k (message length): 4


## All Codewords in the Code

Below are all possible codewords generated by our $[7,4]$ code. Each codeword corresponds to a unique message vector.

In [5]:
decoder.display_code()

Message  ->  Codeword
[0, 0, 0, 0]  ->  [0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1]  ->  [0, 0, 0, 1, 1, 1, 1]
[0, 0, 1, 0]  ->  [0, 0, 1, 0, 0, 1, 1]
[0, 0, 1, 1]  ->  [0, 0, 1, 1, 1, 0, 0]
[0, 1, 0, 0]  ->  [0, 1, 0, 0, 1, 0, 1]
[0, 1, 0, 1]  ->  [0, 1, 0, 1, 0, 1, 0]
[0, 1, 1, 0]  ->  [0, 1, 1, 0, 1, 1, 0]
[0, 1, 1, 1]  ->  [0, 1, 1, 1, 0, 0, 1]
[1, 0, 0, 0]  ->  [1, 0, 0, 0, 1, 1, 0]
[1, 0, 0, 1]  ->  [1, 0, 0, 1, 0, 0, 1]
[1, 0, 1, 0]  ->  [1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1]  ->  [1, 0, 1, 1, 0, 1, 0]
[1, 1, 0, 0]  ->  [1, 1, 0, 0, 0, 1, 1]
[1, 1, 0, 1]  ->  [1, 1, 0, 1, 1, 0, 0]
[1, 1, 1, 0]  ->  [1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1]  ->  [1, 1, 1, 1, 1, 1, 1]


## Decoding Example Received Vectors

We will decode several received vectors, including both valid codewords and vectors with single-bit errors.  
The decoder will return the closest valid message for each received vector.

In [6]:
# Messages and intentionally corrupted versions
examples = [
    ([1, 0, 1, 1], None),          # Valid message, no corruption
    ([1, 0, 1, 1], 4),             # Flip bit 4 → single-bit error
    ([0, 1, 0, 0], 2),             # Flip bit 2
    ([1, 1, 1, 1], 6),             # Flip bit 6
]

for message, flip_index in examples:
    codeword = np.dot(message, G) % 2
    received = codeword.copy()
    if flip_index is not None:
        received[flip_index] ^= 1
    decoded = decoder.decode(received)
    print(f"Message: {message}")
    print(f"Encoded : {codeword.tolist()}")
    print(f"Received: {received.tolist()} (bit {flip_index} flipped)" if flip_index is not None else f"Received: {received.tolist()}")
    print(f"Decoded : {decoded}")
    print("-" * 40)

Message: [1, 0, 1, 1]
Encoded : [1, 0, 1, 1, 0, 1, 0]
Received: [1, 0, 1, 1, 0, 1, 0]
Decoded : [1 0 1 1]
----------------------------------------
Message: [1, 0, 1, 1]
Encoded : [1, 0, 1, 1, 0, 1, 0]
Received: [1, 0, 1, 1, 1, 1, 0] (bit 4 flipped)
Decoded : [1 0 1 1]
----------------------------------------
Message: [0, 1, 0, 0]
Encoded : [0, 1, 0, 0, 1, 0, 1]
Received: [0, 1, 1, 0, 1, 0, 1] (bit 2 flipped)
Decoded : [0 1 0 0]
----------------------------------------
Message: [1, 1, 1, 1]
Encoded : [1, 1, 1, 1, 1, 1, 1]
Received: [1, 1, 1, 1, 1, 1, 0] (bit 6 flipped)
Decoded : [1 1 1 1]
----------------------------------------


## Uncorrectable Example: Two-Bit Error

Now we test the decoder with a received vector that has two bits flipped.  
Since our $[7,4]$ code can only correct single-bit errors, this example demonstrates what happens when more than one error occurs.  
The decoder will still return the closest codeword, but it may not match the original message.  
We also display all codewords at Hamming distance 2 from the received vector to illustrate the decoder's decision.

In [7]:
# Uncorrectable example: flip two bits in a codeword
message = [1, 0, 1, 1]
codeword = np.dot(message, G) % 2
received = codeword.copy()
flip_indices = [2, 5]  # Flip two bits
for idx in flip_indices:
    received[idx] ^= 1

print(f"Original message: {message}")
print(f"Encoded codeword: {codeword.tolist()}")
print(f"Received vector (bits {flip_indices} flipped): {received.tolist()}")

# Compare to codeword for [1, 0, 0, 1]
other_message = [1, 0, 0, 1]
other_codeword = np.dot(other_message, G) % 2
hamming_dist = np.sum(received != other_codeword)
print(f"\nCodeword for message {other_message}: {other_codeword.tolist()}")
print(f"Hamming distance between received and codeword for {other_message}: {hamming_dist}")

decoded = decoder.decode(received)
print(f"\nDecoded message: {decoded}")

Original message: [1, 0, 1, 1]
Encoded codeword: [1, 0, 1, 1, 0, 1, 0]
Received vector (bits [2, 5] flipped): [1, 0, 0, 1, 0, 0, 0]

Codeword for message [1, 0, 0, 1]: [1, 0, 0, 1, 0, 0, 1]
Hamming distance between received and codeword for [1, 0, 0, 1]: 1

Decoded message: [1 0 0 1]
